<img src="https://files.cdn.thinkific.com/file_uploads/133292/images/d57/1b0/1a9/P03.jpg">

`MCSDSC02P03V3______`

`MCSDSC02P03V2______`

`MCSDSELIMC02P03V1130621`

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Проект принят, но общий комментарий — постарайся в следующий раз оставлять ноутбук более-менее чистым для ревью. Выводишь очень много кода, который тестировал в процессе и не закомментировал потом.

<h3><font color="#FF0F00">Комментарий ассистента</font></h3>

Очень понравилась итерация (правда), но есть что доработать. 

Как обычно, заголовки (комментариев) будут трех цветов. <font color="89DD00">Зеленый</font> — все правильно (и если комментария нет, то это также значит, что все правильно). <font color="FFB800">Желтый</font> — можно доработать. <font color="FF0F00">Красный</font> — нужно исправить.

## Постановка задачи

Заказчик исследования — страховая компания. Необходимо выяснить, как разные страховые случаи, возраст клиента и его доход влияют на статус выплаты страховой компенсации.

* Загрузить и преобработать данные:
проверить на наличие пропусков, тип данных, исключить дубликаты, заполнить пропуски, откорректировать отрицательные значения.
* Выделить очевидные однородные причины страховых выплат.
* Категоризировать данные по возрасту и доходу страхуемого потребителя.
* Рассмотреть возможные зависимости выявленных категорий с частотой выплат.

## Импорт данных и библиотек

Импортируем библиотеки, прочитаем файл `insurance.csv` и получим общую информацию о данных.

In [182]:
import pandas as pd
import numpy as np
from nltk.stem import SnowballStemmer
from pymystem3 import Mystem

In [183]:
insur_data=pd.read_csv("../../data/insurance.csv")
#insur_data = pd.read_csv('/Users/elimilo/Desktop/mcs_ds_02/data/insurance.csv')

In [184]:
insur_data.head(10)

,id,perc_of_compensation_paid,age_in_days,age_id,age_group,income,count_3-6_months_late,count_6-12_months_late,count_more_than_12_months_late,cause
0,649,0.001,27384.0,4,Пожилой,NaN,0,0,0,повреждение автомобиля
1,81136,0.124,23735.0,4,пожилой,285140.0,0,0,0,упало дерево на автомобиль
2,70762,1.000,17170.0,3,Зрелый,-186030.0,0,0,0,сломала ногу в отпуске
3,53935,0.198,16068.0,3,Зрелый,-123540.0,0,0,0,хищение автомобиля (каско)
4,15476,0.041,10591.0,2,Молодой,-200020.0,1,0,0,несчастный случай в отпуске
5,64797,0.112,16065.0,3,Зрелый,NaN,0,0,0,врезались в автомобиль
6,67412,0.325,17167.0,3,ЗРЕЛЫЙ,388050.0,6,0,0,травма во время отдыха
7,44241,0.362,20085.0,4,ПОЖИЛОЙ,-99880.0,3,0,0,повреждение автомобиля
8,5069,0.115,23008.0,4,Пожилой,360040.0,0,0,0,сгорела квартира
9,16615,0.021,21906.0,4,Пожилой,510040.0,0,0,0,сгорел дом


In [185]:
insur_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2301 entries, 0 to 2300
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              2301 non-null   int64  
 1   perc_of_compensation_paid       2301 non-null   float64
 2   age_in_days                     2301 non-null   float64
 3   age_id                          2301 non-null   int64  
 4   age_group                       2301 non-null   object 
 5   income                          2236 non-null   float64
 6   count_3-6_months_late           2301 non-null   int64  
 7   count_6-12_months_late          2301 non-null   int64  
 8   count_more_than_12_months_late  2301 non-null   int64  
 9   cause                           2301 non-null   object 
dtypes: float64(3), int64(5), object(2)
memory usage: 179.9+ KB


<h3><font color="#FFB800">Комментарий ассистента 2</font></h3>

Импорт и чтение так и не разделил. И еще все библиотеки, которые ниже используешь, добавляй наверх тоже.

<h3><font color="#FF0F00">Комментарий ассистента</font></h3>

Ты напичкал одну ячейку сразу несколькими по логике командами, воспринимать такое сложно. Импортируй библиотеки в отдельной ячейке, выводи первые 10-15 строк таблицы отдельно и также информацию `info()` тоже стоит в отдельную ячейку поместить.

Столбцы содержат следующую информацию:

* id — ID держателя полиса
* perc_of_compensation_paid — процент выплаченной компенсации
* age_in_days — возраст держателя полиса в днях
* age_id — ID возрастной группы
* age_group — возрастная группа 
* income — доход держателя полиса
* count_3-6_months_late — полис оформлен не позднее 3-6 месяцев
* count_6-12_months_late — полис оформлен не позднее 6-12 месяцев
* count_more_than_12_months_late — полис оформлен больше года назад
* cause — причина страхового случая

### Выводы

* Есть пропуски данных, отрицательные значения в доходах. 
* Наименование возрастных групп с одинаковым age_id различаются по регистру

<h3><font color="#89DD00">Комментарий ассистента 2</font></h3>

Ок

<h3><font color="#FF0F00">Комментарий ассистента</font></h3>

Это не все, что можно заметить из структуры данных. Обрати внимание на age_group и income.

## Предобработка данных

**Обработка пропусков**

Проверим данные на наличие пропусков, просуммировав их.

In [186]:
insur_data.isna().sum()

id                                 0
perc_of_compensation_paid          0
age_in_days                        0
age_id                             0
age_group                          0
income                            65
count_3-6_months_late              0
count_6-12_months_late             0
count_more_than_12_months_late     0
cause                              0
dtype: int64

Заполним пропуски в столбце `income`.

In [187]:
insur_data=insur_data.fillna(insur_data.groupby(['age_id']).transform("median"))
insur_data.head()


,id,perc_of_compensation_paid,age_in_days,age_id,age_group,income,count_3-6_months_late,count_6-12_months_late,count_more_than_12_months_late,cause
0,649,0.001,27384.0,4,Пожилой,162510.0,0,0,0,повреждение автомобиля
1,81136,0.124,23735.0,4,пожилой,285140.0,0,0,0,упало дерево на автомобиль
2,70762,1.000,17170.0,3,Зрелый,-186030.0,0,0,0,сломала ногу в отпуске
3,53935,0.198,16068.0,3,Зрелый,-123540.0,0,0,0,хищение автомобиля (каско)
4,15476,0.041,10591.0,2,Молодой,-200020.0,1,0,0,несчастный случай в отпуске


<h3><font color="#89DD00">Комментарий ассистента 2</font></h3>

Убирай закомментированный код, если больше не нужен.

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Круто, пропуски заполнены верно.

Если значения выборки равномерны и в ней нет выбросов, то среднее подойдет, но когда они есть — среднее значение может сильно сдвинуться вверх или вниз. Для такой выборки лучше использовать медиану. Это стоит учитывать.

Проверяем, что пропусков в `income` не осталось.

In [188]:
insur_data.isna().sum()

id                                0
perc_of_compensation_paid         0
age_in_days                       0
age_id                            0
age_group                         0
income                            0
count_3-6_months_late             0
count_6-12_months_late            0
count_more_than_12_months_late    0
cause                             0
dtype: int64

**Замена типа данных**

Проверяем, что в столбцах `age_in_days` и `income` вещественный тип данных.

In [189]:
insur_data[['age_in_days','income']].dtypes

age_in_days    float64
income         float64
dtype: object

Для этих столбцов нужен целочисленный тип, поэтому переведем их, а также избавимся от отрицательных значений в `income`.

In [190]:
insur_data['income']=insur_data['income'].abs()
insur_data['income'].describe()

count    2.301000e+03
mean     1.948550e+05
std      1.457394e+05
min      0.000000e+00
25%      1.051400e+05
50%      1.640800e+05
75%      2.426300e+05
max      2.700040e+06
Name: income, dtype: float64

In [191]:
insur_data[['age_in_days', 'income']]=insur_data[['age_in_days', 'income']].astype("int64")
insur_data[['age_in_days',"income"]].dtypes

age_in_days    int64
income         int64
dtype: object

<h3><font color="#FFB800">Комментарий ассистента 2</font></h3>

Ок, но постарайся объединять команды в ячейках последовательнее. Например, сначала выводим текущие типы данных, в следующей производим замену, а потом уже проверяем. 

Сейчас не очень понятно по какому принципу ты их объдиняешь. Перевод и проверка 'income' производится во второй, но и в следующей она фигурирует.

<h3><font color="#FFB800">Комментарий ассистента</font></h3>

Не совсем понял для чего здесь `describe()` используешь. Для дохода можно применить цепочку методов `astype('int64').abs()`, а для возраста — только `astype('int64')`.

<h3><font color="#000FFF">Комментарий студента</font></h3>

Перепроверка на отрицательные (min) значения.

**Поиск дубликатов с учетом регистра**

Проверим уникальные категории в столбце `age_group`.

In [192]:
insur_data['age_group'].unique()

array(['Пожилой', 'пожилой', 'Зрелый', 'Молодой', 'ЗРЕЛЫЙ', 'ПОЖИЛОЙ',
       'зрелый', 'молодой', 'МОЛОДОЙ'], dtype=object)

Приведем их к нижнему регистру.

In [193]:
insur_data['age_group']=insur_data['age_group'].str.lower()
insur_data['age_group'].value_counts()

зрелый     1047
пожилой     962
молодой     292
Name: age_group, dtype: int64

<h3><font color="#89DD00">Комментарий ассистента 2</font></h3>

Ок

<h3><font color="#FFB800">Комментарий ассистента</font></h3>

Ок, и можешь еще посчитать здесь размерность каждой группы, будет полезно.

**Обработка дубликатов**

Установим количество явных дубликатов. Если найдутся, то удалим и снова проверим, что их не осталось.

In [194]:
insur_data.duplicated().sum()

7

In [195]:
insur_data=insur_data.drop_duplicates().reset_index(drop=True)

In [196]:
insur_data.duplicated().sum()

0

**Стэмминг**

Посчитаем уникальные значения в столбце `cause`.

In [197]:
insur_data['cause'].value_counts()

кража автомобиля               112
травма во время отдыха         107
сломала ногу в отпуске          95
украли багаж в отпуске          94
травма в отпуске                94
упало дерево на автомобиль      94
сгорел дом                      93
замкнуло электричество дома     91
врезались в автомобиль          91
несчастный случай в отпуске     91
сгорела квартира                91
сотрясение во время отдыха      90
проникли в жилье                89
повреждение автомобиля          89
потоп в квартире                88
хищение автомобиля (каско)      88
несчастный случай на отдыхе     85
повреждение авто                85
кража авто                      82
поломка авто                    81
обокрали дачу                   81
кража в квартире                79
украли авто                     78
взломали квартиру               77
дтп                             76
затопило дом                    73
Name: cause, dtype: int64

Выделим  очевидные разные cтеммы и виды страхования, которые из них вытекают: 
* во время отдыха (отдых отпуск)
* связанные с угоном, кражей повреждение автомобиля (авто дтп)
* страхование недвижимости (квартир дача дом жил)

Напишем функцию, которая принимает основу слова и возвращает все названия категорий с ним. Сначала попробуем стэмминг и если не поймаем «чужих» названий — значит этот метод подойдет.

<h3><font color="#FF0F00">Комментарий ассистента</font></h3>

Стемма — это неизменяемая часть слова.

Лемма — начальная форма слова (примеры для русского языка на платформе).

В описании шага должно быть «Выделим очевидные разные **cтеммы**», это наша ошибка, надо поправить.

И еще через дефис тоже неправильно писать это, лучше «выделим очевидные разные cтеммы и виды страхования, которые из них вытекают».

In [198]:
def find_lem(word):
    word_data=insur_data[insur_data['cause'].str.contains(word)]
    print(word_data['cause'].value_counts())

find_lem("авто")

кража автомобиля              112
упало дерево на автомобиль     94
врезались в автомобиль         91
повреждение автомобиля         89
хищение автомобиля (каско)     88
повреждение авто               85
кража авто                     82
поломка авто                   81
украли авто                    78
Name: cause, dtype: int64


In [199]:
russian_stemmer = SnowballStemmer('russian')

In [200]:
''' 
def find_stem(df):
    categories=df.unique().tolist()
    print(categories)
    stemmed_words=[]
    for category in categories:
        category=category.split(" ")
        print(category)
        for word in category:
            stemmed_word=russian_stemmer.stem(word)
            print(word, " : ", stemmed_word)
            stemmed_words.append(stemmed_word)
    return stemmed_words
  
a=find_stem(insur_data['cause'])
a=pd.DataFrame(a)
a[0].value_counts()

'''   

def do_stemming(word_stem):
    queries = insur_data['cause'].unique().tolist()
    print(word_stem)
    for query in queries:
        
        for word in query.split():
            
            stemmed_word = russian_stemmer.stem(word)
            
            
            for stemms in word_stem:
                
                if stemmed_word == stemms:
                    print(query)
              
word_stem_auto=['авт','автомобил', "дтп"]
word_stem_r_estate=['квартир','дом', "жил"]
word_stem_holiday=['отдых','отпуск']

do_stemming(word_stem_auto)
do_stemming(word_stem_r_estate)
do_stemming(word_stem_holiday)

['авт', 'автомобил', 'дтп']
повреждение автомобиля
упало дерево на автомобиль
хищение автомобиля (каско)
врезались в автомобиль
кража автомобиля
украли авто
повреждение авто
кража авто
поломка авто
дтп
['квартир', 'дом', 'жил']
сгорела квартира
сгорел дом
потоп в квартире
проникли в жилье
замкнуло электричество дома
взломали квартиру
затопило дом
кража в квартире
['отдых', 'отпуск']
сломала ногу в отпуске
несчастный случай в отпуске
травма во время отдыха
украли багаж в отпуске
сотрясение во время отдыха
несчастный случай на отдыхе
травма в отпуске


<h3><font color="#000FFF">Комментарий студента 2</font></h3>

Переформатировал для наглядности, одиночные стеммы мало что показывают.

<h3><font color="#FF0F00">Комментарий ассистента 2</font></h3>

А почему ты ей весь столбец передаешь? Тебе же надо проверить вхождение стеммы в категорию, как ты сделал в первой своей функции.

```
queries = ins['cause'].unique().tolist()

def do_stemming(word_stem):

    for query in queries:

        for word in query.split():
            stemmed_word = russian_stemmer.stem(word)
            if stemmed_word == word_stem:
                print(query)

do_stemming('авто')
```

<h3><font color="#000FFF">Комментарий студента</font></h3>

Выделил визуально "стемы", но непонимаю - как это можно теперь использовать (механически)? Руками подсталяем в def lem(row)?
- авт             4
- квартир         4
- отпуск          4
- краж            3
- отдых           3
- дом             3

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Круто, ты справился с одной из самых сложных частей проекта. Расскажу дополнительно о решении, которое мы ожидаем от студентов.

В Python для стемминг есть специальная библиотека NLTK

```
from nltk.stem import SnowballStemmer
russian_stemmer = SnowballStemmer('russian')
```

которая содержит стеммеры (объекты, определяющие правила нахождения стемов).

В первую очередь тебе необходимо сформировать список текущих уникальных категорий с помощью методов `unique()` и `tolist()`.

Далее необходимо написать функцию, которая принимает основу слова, а возвращает все категории с ней. Ты сможешь добраться до нее с помощью двух циклов `for category in categories` и `for word in category` (когда будешь основу в категории, раздели с помощью метода `split()` все слова в ней).

Далее сохраняй stemmed_word с помощью русского стеммера и метода `stem()` (методу передавай итератор word) и проверяй с помощью условия равно ли stemmed_word той основе, которую ты передаешь функции.

Для нас важно, что ты понял как работает эта библиотека.

Так же стоит подробнее объяснить какие виды страхования вытекают из стемм. Их ты дальше будешь использовать в сводной таблице.

**Категоризация данных**

Cловарь — распространенный способ хранения информации, который помогает убрать текстовые параметры из основной таблицы и увеличить эффективность работы с данными. Выделим словарь для возрастных групп.

In [201]:
Dict_age=pd.DataFrame(insur_data[['age_group', 'age_id']])
Dict_age

,age_group,age_id
0,пожилой,4
1,пожилой,4
2,зрелый,3
3,зрелый,3
4,молодой,2
...,...,...
2289,зрелый,3
2290,зрелый,3
2291,пожилой,4
2292,пожилой,4


<h3><font color="#FF0F00">Комментарий ассистента 2</font></h3>

Посмотрел конспект, на который я сослался?) У каждой категории есть id:

Пожилой — id 4

Зрелый — id 3

Молодой — id 2

И вот тебе нужно получить 2 этих столбца из основной таблицы.

<h3><font color="#FF0F00">Комментарий ассистента</font></h3>

Выделить словарь таким образом нельзя, потому что у каждой категории в основной таблице уже есть id. Посмотри на платформе конспект «Доступ к значениям DataFrame», там показано как несколько столбцов получить.

После этой операции текстовые значения age_group в основной таблице можно удалить и использовать только их индекс, что ускорит обработку данных и визуальную работу с таблицей, так как категории могут быть разной длины.

<h3><font color="#000FFF">Комментарий студента</font></h3>

Понять, что требуется в задании сложнее чем сделать.

Далее, опираясь на результат прошлого стэмминга, — категоризируем причины выплаты страховых платежей.

In [202]:
m = Mystem()
print(insur_data['cause'])

0            повреждение автомобиля
1        упало дерево на автомобиль
2            сломала ногу в отпуске
3        хищение автомобиля (каско)
4       несчастный случай в отпуске
                   ...             
2289                            дтп
2290               кража в квартире
2291               проникли в жилье
2292                  обокрали дачу
2293         повреждение автомобиля
Name: cause, Length: 2294, dtype: object


In [203]:
def lem(row):
        l = m.lemmatize(row)
        print(l)
        if 'отдых' in l or 'отпуск' in l:
            return 'Страхование на отдыхе'
        elif 'автомобиль' in l or 'авто' in l or 'дтп' in l:
            return 'Страхование личного транспорта'
        elif 'квартира' in l or 'дача' in l or 'дом' in l or 'жилье' in l:
            return 'Страхование недвижимости'
            
insur_data['cause'] = insur_data['cause'].apply(lem)

['повреждение', ' ', 'автомобиль', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['хищение', ' ', 'автомобиль', ' (', 'каско', ')\n']
['несчастный', ' ', 'случай', ' ', 'в', ' ', 'отпуск', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['сгорать', ' ', 'квартира', '\n']
['сгорать', ' ', 'дом', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['кража', ' ', 'автомобиль', '\n']
['украсть', ' ', 'авто', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['повреждение', ' ', 'авто', '\n']
['проникать', ' ', 'в', ' ', 'жилье', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['украсть', ' ', 'авто', '\n']
['сгорать', ' ', 'дом', '\n']
[

['кража', ' ', 'в', ' ', 'квартира', '\n']
['сгорать', ' ', 'квартира', '\n']
['украсть', ' ', 'авто', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['кража', ' ', 'в', ' ', 'квартира', '\n']
['взламывать', ' ', 'квартира', '\n']
['поломка', ' ', 'авто', '\n']
['кража', ' ', 'авто', '\n']
['кража', ' ', 'автомобиль', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['повреждение', ' ', 'авто', '\n']
['кража', ' ', 'авто', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['обкрадывать', ' ', 'дача', '\n']
['сотрясение', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['поломка', ' ', 'авто', '\n']
['кража', ' ', 'авто', '\n']
['повреждение',

['дтп', '\n']
['сгорать', ' ', 'дом', '\n']
['кража', ' ', 'автомобиль', '\n']
['кража', ' ', 'автомобиль', '\n']
['кража', ' ', 'автомобиль', '\n']
['затапливать', ' ', 'дом', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['кража', ' ', 'автомобиль', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['несчастный', ' ', 'случай', ' ', 'на', ' ', 'отдых', '\n']
['поломка', ' ', 'авто', '\n']
['замыкать', ' ', 'электричество', ' ', 'дом', '\n']
['сгорать', ' ', 'квартира', '\n']
['кража', ' ', 'в', ' ', 'квартира', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['несчастный', ' ', 'случай', ' ', 'на', ' ', 'отдых', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['дтп', '\n']
['замыкать', ' ', 'электричество', ' ', 'дом', '\n']
['хищение', ' ', 'автомобиль', ' (', 'каско', ')\n']
['взламывать', ' ', 'квартира', '\n']
['повреждение', ' ', 'авто', '\n']
['сгорать', ' ', 'квартира', '\n']
['травма', ' ', 

['несчастный', ' ', 'случай', ' ', 'на', ' ', 'отдых', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['сгорать', ' ', 'квартира', '\n']
['кража', ' ', 'автомобиль', '\n']
['сгорать', ' ', 'дом', '\n']
['украсть', ' ', 'авто', '\n']
['несчастный', ' ', 'случай', ' ', 'в', ' ', 'отпуск', '\n']
['дтп', '\n']
['проникать', ' ', 'в', ' ', 'жилье', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['сгорать', ' ', 'дом', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['взламывать', ' ', 'квартира', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['затапливать', ' ', 'дом', '\n']
['поломка', ' ', 'авто', '\n']
['кража', ' ', 'автомобиль', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['кража', ' ', 'авто', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['проникать', ' ', 'в', ' ', 'жилье', '\n']
['со

['замыкать', ' ', 'электричество', ' ', 'дом', '\n']
['затапливать', ' ', 'дом', '\n']
['несчастный', ' ', 'случай', ' ', 'на', ' ', 'отдых', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['проникать', ' ', 'в', ' ', 'жилье', '\n']
['кража', ' ', 'в', ' ', 'квартира', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['несчастный', ' ', 'случай', ' ', 'в', ' ', 'отпуск', '\n']
['замыкать', ' ', 'электричество', ' ', 'дом', '\n']
['сгорать', ' ', 'квартира', '\n']
['дтп', '\n']
['проникать', ' ', 'в', ' ', 'жилье', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['несчастный', ' ', 'случай', ' ', 'в', ' ', 'отпуск', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['затапливать', ' ', 'дом', '\n']
['несчастный', ' ', 'случай', ' ', 'на', ' ', 'отдых', 

...

['затапливать', ' ', 'дом', '\n']
['хищение', ' ', 'автомобиль', ' (', 'каско', ')\n']
['проникать', ' ', 'в', ' ', 'жилье', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['дтп', '\n']
['кража', ' ', 'в', ' ', 'квартира', '\n']
['повреждение', ' ', 'авто', '\n']
['сгорать', ' ', 'дом', '\n']
['повреждение', ' ', 'авто', '\n']
['украсть', ' ', 'авто', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['проникать', ' ', 'в', ' ', 'жилье', '\n']
['украсть', ' ', 'авто', '\n']
['сгорать', ' ', 'дом', '\n']
['взламывать', ' ', 'квартира', '\n']
['хищение', ' ', 'автомобиль', ' (', 'каско', ')\n']
['сотрясение', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['повреждение', ' ', 'авто', '\n']
['сгорать', ' ', 'дом', '\n']
['хищение', ' ', 'автомобиль', ' (', 'каско', ')\n']
['замыкать', ' ', 'электричество', ' ', 'дом', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['травма', ' ', 'в', ' ', 'отпуск', '\n']
['кража', ' ', 'авто', '\n']
['несчас

['кража', ' ', 'автомобиль', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['взламывать', ' ', 'квартира', '\n']
['украсть', ' ', 'авто', '\n']
['кража', ' ', 'автомобиль', '\n']
['сгорать', ' ', 'квартира', '\n']
['хищение', ' ', 'автомобиль', ' (', 'каско', ')\n']
['сгорать', ' ', 'квартира', '\n']
['замыкать', ' ', 'электричество', ' ', 'дом', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['украсть', ' ', 'авто', '\n']
['дтп', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['кража', ' ', 'в', ' ', 'квартира', '\n']
['замыкать', ' ', 'электричество', ' ', 'дом', '\n']
['украсть', ' ', 'авто', '\n']
['украсть', ' ', 'авто', '\n']
['травма', ' ', 'в', ' ', 'отпуск', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
[

['повреждение', ' ', 'авто', '\n']
['обкрадывать', ' ', 'дача', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['дтп', '\n']
['повреждение', ' ', 'авто', '\n']
['замыкать', ' ', 'электричество', ' ', 'дом', '\n']
['несчастный', ' ', 'случай', ' ', 'на', ' ', 'отдых', '\n']
['обкрадывать', ' ', 'дача', '\n']
['поломка', ' ', 'авто', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['сотрясение', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['повреждение', ' ', 'авто', '\n']
['украсть', ' ', 'авто', '\n']
['поломка', ' ', 'авто', '\n']
['сотрясение', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['несчастный', ' ', 'случай', ' ', 'в', ' ', 'отпуск', '\n']
['травма', ' ', 'в', ' ', 'отпуск', '\n']
['взламывать', ' ', 'квартира', '\n']
['украсть', ' ', 'авто', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['взламывать', ' ', 'квартира', '\n']
['травма', ' ', 'в', ' ', 'отпуск', '\n']
['поломка', ' ', 'авто', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['трав

['кража', ' ', 'автомобиль', '\n']
['сгорать', ' ', 'дом', '\n']
['кража', ' ', 'в', ' ', 'квартира', '\n']
['затапливать', ' ', 'дом', '\n']
['взламывать', ' ', 'квартира', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['кража', ' ', 'автомобиль', '\n']
['хищение', ' ', 'автомобиль', ' (', 'каско', ')\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['хищение', ' ', 'автомобиль', ' (', 'каско', ')\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['травма', ' ', 'в', ' ', 'отпуск', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['поломка', ' ', 'авто', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['обкрадывать', ' ', 'дача', '\n']
['сгорать', ' ', 'квартира', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['взламывать', ' ', 'квартира', 

['сгорать', ' ', 'квартира', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['кража', ' ', 'авто', '\n']
['замыкать', ' ', 'электричество', ' ', 'дом', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['травма', ' ', 'в', ' ', 'отпуск', '\n']
['сгорать', ' ', 'дом', '\n']
['проникать', ' ', 'в', ' ', 'жилье', '\n']
['кража', ' ', 'авто', '\n']
['повреждение', ' ', 'авто', '\n']
['сгорать', ' ', 'дом', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['травма', ' ', 'в', ' ', 'отпуск', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['сотрясение', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['кража', ' ', 'автомобиль', '\n']
['несчастный', ' ', 'случай', ' ', 'в', ' ', 'отпуск', '\n']
['травма', ' ', 'в', ' ', 'отпуск', '\n']
['сгорать', ' ', 'дом', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['повреждение', ' ', 'авто', '\n']
['несчастный', ' ', 'случай', ' ', 'в', ' '

In [204]:
insur_data['cause'].value_counts()

Страхование личного транспорта    876
Страхование недвижимости          762
Страхование на отдыхе             656
Name: cause, dtype: int64

<h3><font color="#89DD00">Комментарий ассистента 2</font></h3>

Здесь все ок.

<h3><font color="#FF0F00">Комментарий ассистента</font></h3>

С такой реализацией я уже не согласен. Для решения этот задачи есть библиотека pymystem3

```
from pymystem3 import Mystem
m = Mystem()
```

которая в отличие от NLTK выдает *список* лемматизированных слов (сведенных к словарной форме), а не строку. 

Для этого необходимо написать функцию лемматизации

```
def lem(row):
    l = m.lemmatize(row)
    if 'word_1' in l or 'word_2' in l:
        return 'new_category'
    ...
```

и применить ее через метод `apply()` к столбцу с причинами.

```
df['column_1'] = df['column_1'].apply(lem)
```

Так же обрати внимание на итоговые категории. Да, ты правильно выделил эти 3 группы, осталось только назвать их правильнее — по виду страхования.

<h3><font color="#000FFF">Комментарий студента</font></h3>

Совсем не реагирует на частичные 'авто', только на стандартную форму слова.

---

Категоризируем статус выплаты компенсации — 0 выплат, выплачена часть, полностью выплачена.

In [205]:
'''
def paid():
    insur_data.loc[insur_data['perc_of_compensation_paid'] == 1, 'paid'] = "paid"
    insur_data.loc[insur_data['perc_of_compensation_paid'] <= 0.00, 'paid'] = "unpaid"
    insur_data.loc[(insur_data['perc_of_compensation_paid'] < 1) & (insur_data['perc_of_compensation_paid'] >0.00) , 'paid'] = "part"
paid()
insur_data['paid'].value_counts()
 '''

def paid_2(row):
        if row == 1:
            return 'paid'
        elif row == 0:
            return "unpaid"
        else:
            return "part"
insur_data['paided'] = insur_data['perc_of_compensation_paid'].apply(paid_2) 
insur_data['paided'].value_counts()

part      1788
paid       356
unpaid     150
Name: paided, dtype: int64

<h3><font color="#89DD00">Комментарий ассистента 2</font></h3>

Отлично.

<h3><font color="#FF0F00">Комментарий ассистента</font></h3>

Правильный результат, круто, но по реализации мы ожидаем увидеть применение функции через метод `apply()`. Нам важно увидеть, что ты его освоил. Прочитай, пожалуйста, про него в конспекте по категоризации и напиши новое решение `paid_2()`. 

Внутри функции у тебя должно быть простое условие: если выплачено 1.0, то возращай 'paid'. Задавай вопросы в ветку категоризации, если будет что-то непонятно.

Категоризируем `income`, разбив данные по квантилям.

In [206]:
insur_data['income']

0       162510
1       285140
2       186030
3       123540
4       200020
         ...  
2289    180070
2290    180070
2291    162510
2292    162510
2293    162510
Name: income, Length: 2294, dtype: int64

In [207]:
insur_data['quant']=pd.qcut(insur_data['income'], 5, labels=["low","lower-med", "medium","higher-med" , "high"])
insur_data

,id,perc_of_compensation_paid,age_in_days,age_id,age_group,income,count_3-6_months_late,count_6-12_months_late,count_more_than_12_months_late,cause,paided,quant
0,649,0.001,27384,4,пожилой,162510,0,0,0,Страхование личного транспорта,part,medium
1,81136,0.124,23735,4,пожилой,285140,0,0,0,Страхование личного транспорта,part,high
2,70762,1.000,17170,3,зрелый,186030,0,0,0,Страхование на отдыхе,paid,medium
3,53935,0.198,16068,3,зрелый,123540,0,0,0,Страхование личного транспорта,part,lower-med
4,15476,0.041,10591,2,молодой,200020,1,0,0,Страхование на отдыхе,part,higher-med
...,...,...,...,...,...,...,...,...,...,...,...,...
2289,16440,0.474,18628,3,зрелый,180070,0,0,0,Страхование личного транспорта,part,medium
2290,109416,0.413,17525,3,зрелый,180070,0,0,0,Страхование недвижимости,part,medium
2291,4268,0.109,21906,4,пожилой,162510,0,0,0,Страхование недвижимости,part,medium
2292,85708,0.036,25925,4,пожилой,162510,0,0,0,Страхование недвижимости,part,medium


<h3><font color="#89DD00">Комментарий ассистента 2</font></h3>

Ок.

<h3><font color="#FF0F00">Комментарий ассистента</font></h3>

Очень сложно, здесь стоит использовать метод пандас qcut. Он принимает столбец, по которому категоризируем, количество корзин и лэйблы. С ним у тебя каждая группа должна получится одинаковой размерности. Он тебе понравится и все упростит, переделай с ним.

## Результаты

Подготовим 3 таблицы и изучим результат.

<h3><font color="#89DD00">Комментарий ассистента 2</font></h3>

Да, стало лучше.

<h3><font color="#FFB800">Комментарий ассистента</font></h3>

Круто, ты самостоятельно разобрался с аргументами `pivot_table()`. У нас результаты незначительно отличаются, но это из-за разной категоризации.

И можешь еще наш конспект по изменению типов данных посмотреть, там тоже есть кратко об аргументах `pivot_table()`. Кстати, и для aggfunc можешь просто 'count' использовать, пропусков у тебя не должно быть.

*ответьте на вопрос — есть ли зависимость между возрастной группой и статусом выплаты?*

In [208]:
pd.pivot_table(insur_data, index =['age_group'], columns=["paided"], values=["id"], 
               aggfunc='count').apply(lambda x: (x / x.sum())*100, axis=1).round(0)

id             
paided     paid  part unpaid
age_group                   
зрелый     10.0  85.0    6.0
молодой    18.0  71.0   11.0
пожилой    21.0  73.0    6.0

*есть ли зависимость между причиной страхового случая и статусом выплаты?*

In [209]:
pd.pivot_table(insur_data, index = ['cause'], columns=["paided"], values=["id"], 
               aggfunc='count').apply(lambda x: (x / x.sum())*100, axis=1).round(0)

id             
paided                          paid  part unpaid
cause                                            
Страхование личного транспорта  21.0  73.0    6.0
Страхование на отдыхе           12.0  82.0    6.0
Страхование недвижимости        13.0  80.0    7.0

*есть ли зависимость между доходом клиента и статусом выплаты?*

In [210]:
pd.pivot_table(insur_data, index = ['quant'], columns=["paided"], values=["id"], 
               aggfunc='count').apply(lambda x: (x / x.sum())*100, axis=1).round(0)

id             
paided      paid  part unpaid
quant                        
low         20.0  70.0   10.0
lower-med   15.0  78.0    7.0
medium      12.0  81.0    7.0
higher-med  15.0  81.0    4.0
high        14.0  80.0    5.0

<h3><font color="#89DD00">Комментарий ассистента 2</font></h3>

С долями станет лучше, да. Для aggfunc можно свою функцию применить.

<h3><font color="#000FFF">Комментарий студента</font></h3>

Нашел вот такой костыль, напрямую со сводной таблицей работать затрудняюсь.

---

<h3><font color="#FFB800">Комментарий ассистента</font></h3>

Попробуй ради практики пересобрать свои таблицы таким образом, чтобы столбцами были 3 статуса выплата, а строками — результат твоей категоризации. Мне сложновато воспринимать сейчас финальный результат, но по идее он правильный.

И еще доход клиента "недокатегоризирован" сейчас, но ты решишь это с помощью лэйблов qcut. Называй как-нибудь low, lower-middle и тд.

<h3><font color="#000FFF">Комментарий студента</font></h3>

Без перевода в % соотношение таблица мне кажется совсем не выразительной, но в aggfunc не нашел, а в ручную не соображу как.

---

*  Молодым людям отказывают в 2 раза чаще зрелых и пожилых.
*  Сильной зависимости между причиной страхового случая и статусом выплаты нет, но в 2 раза чаще выплачивают полную страховку на автомобиль. 
* Наиболее бедным слоям страховку выплачивают в 2 раза реже богатых, но если дойдет до выплат, то бедным немногим чаще выплачивают полную стоимость.